В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import os

from mlxtend.plotting import plot_decision_regions

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [32]:
%reload_ext autoreload
%autoreload 2

from process_bank_churn import preprocess_data

In [33]:
raw_df = pd.read_csv('train.csv', index_col=0)
data = preprocess_data(raw_df, target_col='Exited')

In [34]:
raw_df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,,,
0,15779985.0,Nwankwo,678.0,France,Male,29.0,4.0,0.00,3.0,1.0,0.0,180626.36,0.0
1,15650086.0,Ch'in,687.0,France,Female,34.0,1.0,0.00,2.0,0.0,1.0,63736.17,0.0
2,15733602.0,Thompson,682.0,France,Female,52.0,6.0,0.00,3.0,0.0,0.0,179655.87,1.0
3,15645794.0,Macleod,753.0,Germany,Male,44.0,6.0,83347.25,2.0,1.0,0.0,161407.48,0.0
4,15633840.0,Hsia,544.0,Germany,Female,55.0,0.0,107747.57,1.0,1.0,0.0,176580.86,1.0


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [35]:
# Створення і навчання моделі з параметрами за замовченням
knn_default = KNeighborsClassifier()
knn_default.fit(data['train_X'], data['train_y'])

KNeighborsClassifier()

In [38]:
# Ймовірності для AUROC
y_train_proba = knn_default.predict_proba(data['train_X'])[:, 1]
y_val_proba = knn_default.predict_proba(data['val_X'])[:, 1]

In [40]:
# Обчислення AUROC
train_auc = roc_auc_score(data['train_y'], y_train_proba)
val_auc = roc_auc_score(data['val_y'], y_val_proba)

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

Train AUROC: 0.9559
Validation AUROC: 0.8526


###### Модель kNN з параметрами за замовченням показує високу якість на тренувальних даних (AUROC ≈ 0.956) та помітно нижчу — на валідаційних (≈ 0.853). Це свідчить про перенавчання моделі, тобто високу дисперсію (high variance). Модель ще не ідеальна, і є потенціал для покращення через підбір гіперпараметрів, зокрема n_neighbors.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [41]:
from sklearn.model_selection import GridSearchCV

# Підбір параметра n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}
knn = KNeighborsClassifier()

In [42]:
grid_knn = GridSearchCV(knn, params_knn, cv=5, scoring='roc_auc')
grid_knn.fit(data['train_X'], data['train_y'])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             scoring='roc_auc')

In [43]:
# Найкращий параметр і модель
print("Best parameters:", grid_knn.best_params_)
knn_best = grid_knn.best_estimator_

Best parameters: {'n_neighbors': np.int64(19)}


In [44]:
# Оцінка AUROC
y_train_proba_best = knn_best.predict_proba(data['train_X'])[:, 1]
y_val_proba_best = knn_best.predict_proba(data['val_X'])[:, 1]

train_auc_best = roc_auc_score(data['train_y'], y_train_proba_best)
val_auc_best = roc_auc_score(data['val_y'], y_val_proba_best)

print(f"Train AUROC (best): {train_auc_best:.4f}")
print(f"Validation AUROC (best): {val_auc_best:.4f}")


Train AUROC (best): 0.9227
Validation AUROC (best): 0.8908


###### Після підбору параметра n_neighbors за допомогою GridSearchCV, якість моделі значно покращилася на валідаційній вибірці (AUROC виріс з 0.8526 до 0.8908), при цьому перенавчання помітно зменшилося. high variance значно зменшилась, і модель стала більш узагальнюючою.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import time

In [48]:
# Ініціалізація моделі
dt = DecisionTreeClassifier(random_state=42)

In [49]:
# Налаштування параметрів для пошуку
param_grid = {
    'max_depth': range(1, 21, 2),
    'max_leaf_nodes': range(2, 11)
}

In [51]:
# Налаштування GridSearchCV
grid_search = GridSearchCV(dt, param_grid, cv=3, scoring='roc_auc', verbose=1)

# Початок вимірювання часу
start_time = time.time()

# Навчання моделі
grid_search.fit(data['train_X'], data['train_y'])

# Кінець вимірювання часу
end_time = time.time()

# Час пошуку
print(f"Час пошуку гіперпараметрів: {end_time - start_time:.2f} секунд")

# Найкращі гіперпараметри
print(f"Найкращі параметри: {grid_search.best_params_}")

# Збереження найкращої моделі
dt_best = grid_search.best_estimator_

Fitting 3 folds for each of 90 candidates, totalling 270 fits
Час пошуку гіперпараметрів: 11.79 секунд
Найкращі параметри: {'max_depth': 5, 'max_leaf_nodes': 10}


In [52]:
# Оцінка якості на тренувальній і валідаційній вибірці
y_train_pred = dt_best.predict_proba(data['train_X'])[:, 1]
y_val_pred = dt_best.predict_proba(data['val_X'])[:, 1]

train_auc = roc_auc_score(data['train_y'], y_train_pred)
val_auc = roc_auc_score(data['val_y'], y_val_pred)

print(f"Train AUROC (dt_best): {train_auc:.4f}")
print(f"Validation AUROC (dt_best): {val_auc:.4f}")

Train AUROC (dt_best): 0.9015
Validation AUROC (dt_best): 0.9002


###### Після підбору гіперпараметрів для Decision Tree за допомогою GridSearchCV модель показала високу якість і стабільність на тренувальних і валідаційних даних. Вона краща за попередню версію, побудовану вручну, і навіть перевершує оптимізований kNN. Таким чином, ця модель — найкраща з-поміж протестованих у цьому завданні. і + набагато менше часу пішло на виконання.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [56]:
from sklearn.model_selection import RandomizedSearchCV

In [57]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [58]:
# Фіксуємо random_state
random_state = 42

# Ініціалізація моделі
dt = DecisionTreeClassifier(random_state=random_state)

# RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=params_dt,
    n_iter=40,
    scoring='roc_auc',
    cv=3,
    random_state=random_state,
    verbose=2,
    n_jobs=-1
)

# Вимірюємо час
start_time = time.time()

search.fit(data['train_X'], data['train_y'])

end_time = time.time()
print(f" Час пошуку гіперпараметрів: {end_time - start_time:.2f} секунд")

# Найкращі параметри
print(" Найкращі параметри:", search.best_params_)

# Збереження моделі
dt_random_search_best = search.best_estimator_

# Прогнози
y_train_proba = dt_random_search_best.predict_proba(data['train_X'])[:, 1]
y_val_proba = dt_random_search_best.predict_proba(data['val_X'])[:, 1]

# Обчислення AUROC
train_auc = roc_auc_score(data['train_y'], y_train_proba)
val_auc = roc_auc_score(data['val_y'], y_val_proba)

print(f"Train AUROC (RandomSearch): {train_auc:.4f}")
print(f"Validation AUROC (RandomSearch): {val_auc:.4f}")

Fitting 3 folds for each of 40 candidates, totalling 120 fits
 Час пошуку гіперпараметрів: 8.69 секунд
 Найкращі параметри: {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
Train AUROC (RandomSearch): 0.9169
Validation AUROC (RandomSearch): 0.9166


###### Модель, знайдена з RandomizedSearch, показала кращу якість на валідаційній вибірці, ніж GridSearch. Це свідчить, що випадковий пошук охопив кращу комбінацію гіперпараметрів, яку не охопив GridSearch через обмежену сітку. Різниця між Train та Validation AUROC дуже маленька (≈ 0.0003), що вказує на гарне узагальнення моделі. Це не high variance і не high bias — модель добре збалансована.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [59]:
from process_bank_churn import preprocess_new_data

In [60]:
# Завантажуємо test.csv
test_df = pd.read_csv("test.csv", index_col=0)

# Визначаємо ті самі вхідні колонки, що й у train
exclude_cols = {'id', 'RowNumber', 'CustomerId', 'Surname'}
input_cols = [col for col in test_df.columns if col not in exclude_cols]
test_inputs = test_df[input_cols]

# Використовуємо артефакти, отримані з preprocess_data
X_test = preprocess_new_data(test_inputs, data['preprocessor'])

C:\Users\linam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [61]:
y_test_proba = dt_random_search_best.predict_proba(X_test)[:, 1]

In [62]:
# Формуємо DataFrame з результатами
output_df = pd.DataFrame({
    'id': test_df.index,  # Якщо є індекси або унікальні ідентифікатори
    'Prediction': y_test_proba  # Ймовірність передбаченого класу 1
})

# Зберігаємо результати у файл
output_df.to_csv('submission.csv', index=False)